<a href="https://colab.research.google.com/github/TynK-M/nursing-fall-risk-assessment/blob/main/nursing_fall_risk_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Morse Fall Scale

The MFS (Morse Fall Scale) is considered one of the fastest methods to assessing a patient probability of falling.

### Risk Categories

In [ ]:
from enum import Enum

In [ ]:
class MorseFallRisk(Enum):
  LOW = 1
  MEDIUM = 2
  HIGH = 3

  def contains(self, score: int) -> bool:
    if self is MorseFallRisk.LOW:
      return score < 25
    elif self is MorseFallRisk.MEDIUM:
      return 25 <= score <= 45
    elif self is MorseFallRisk.HIGH:
      return score > 45
    else:
      raise NotImplementedError(f"No rule for {self}")

  @classmethod
  def from_score(cls, score: int) -> "MorseFallRisk":
    for tier in cls:
      if tier.contains(score):
        return tier.name
    raise ValueError(f"Invalid Morse Fall Scale score: {score}")

## Variables

### Ambulatory Aid

In [ ]:
low_ambulatory_aid = [
    "bed rest",
    "nurse assist",
]

medium_ambulatory_aid = [
    "crutches",
    "cane",
    "walker",
]

high_ambulatory_aid = [
    "furniture",
]

### Gait or Transfering

In [ ]:
low_gait_or_transfering = [
    "normal",
    "bed rest",
    "immobile",
]

medium_gait_or_transfering = [
    "weak",
]

high_gait_or_transfering = [
    "impaired",
]

## Morse Fall Scale

In [ ]:
def morse_fall_scale(falling_history: bool, secondary_diagnosis: bool, ambulatory_aid: str, iv_or_heparin_lock: bool, gait_or_transfering: str, does_forget_limitations: bool) -> int:
    score = 0

    score += 25 if falling_history else 0
    score += 15 if secondary_diagnosis else 0

    if ambulatory_aid in high_ambulatory_aid:
        score += 30
    elif ambulatory_aid in medium_ambulatory_aid:
        score += 15

    score += 20 if iv_or_heparin_lock else 0

    if gait_or_transfering in high_gait_or_transfering:
        score += 20
    elif gait_or_transfering in medium_gait_or_transfering:
        score += 10

    score += 15 if does_forget_limitations else 0

    return score

## Dataset

The dataset is generated and does not use real patient data.

In [ ]:
import pandas as pd
import random

In [ ]:
NUMBER_OF_SIMULATED_PATIENTS = 200

In [ ]:
def generate_patient() -> dict:
  return {
      "falling_history": random.choice([True, False]),
      "secondary_diagnosis": random.choice([True, False]),
      "ambulatory_aid": random.choice([None] + low_ambulatory_aid + medium_ambulatory_aid + high_ambulatory_aid),
      "iv_or_heparin_lock": random.choice([True, False]),
      "gait_or_transfering": random.choice([None] + low_gait_or_transfering + medium_gait_or_transfering + high_gait_or_transfering),
      "does_forget_limitations": random.choice([True, False])
  }

In [ ]:
data = [generate_patient() for _ in range(NUMBER_OF_SIMULATED_PATIENTS)]

In [ ]:
df = pd.DataFrame(data)

## Output

In [ ]:
df["morse_score"] = df.apply(
    lambda row: morse_fall_scale(
        row["falling_history"],
        row["secondary_diagnosis"],
        row["ambulatory_aid"],
        row["iv_or_heparin_lock"],
        row["gait_or_transfering"],
        row["does_forget_limitations"]
    ),
    axis=1
)

df["risk_level"] = df["morse_score"].apply(MorseFallRisk.from_score)

In [ ]:
df

## Karnaugh map

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import itertools

Simplified Morse Fall Scale scoring for easier plotting of the Karnaugh map

In [ ]:
LOW_THRESHOLD = 25
HIGH_THRESHOLD = 45

In [ ]:
def morse_fall_score(falling_history, secondary_diagnosis, high_ambulatory_aid,
                     iv_or_heparin_lock, high_gait_or_transfering, does_forget_limitations):
    score = 0
    score += 25 if falling_history else 0
    score += 15 if secondary_diagnosis else 0
    score += 30 if high_ambulatory_aid else 0
    score += 20 if iv_or_heparin_lock else 0
    score += 20 if high_gait_or_transfering else 0
    score += 15 if does_forget_limitations else 0
    return score

In [ ]:
def get_risk_color(score):
    if score < LOW_THRESHOLD:
        return '#a1d99b'  # green for low risk
    elif score <= HIGH_THRESHOLD:
        return '#fec44f'  # yellow for medium risk
    else:
        return '#de2d26'  # red for high risk

gray = ['000', '001', '011', '010', '110', '111', '101', '100']

scores = []
for bits in itertools.product([0,1], repeat=6):
    score = morse_fall_score(*bits)
    scores.append(score)

submap_labels = {
    0: 'A=0, B=0',
    1: 'A=0, B=1',
    2: 'A=1, B=0',
    3: 'A=1, B=1'
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle("6-Variable Karnaugh Map for Morse Fall Scale", fontsize=16)

for idx, ax in enumerate(axes.flat):
    ax.axis('off')
    table_data = []
    cell_colors = []
    for r_idx, r in enumerate(gray):
        row = []
        color_row = []
        for c_idx, c in enumerate(gray):
            a, b = (idx >> 1) & 1, idx & 1
            c_bit, d_bit, e_bit, f_bit = int(r[1]), int(r[2]), int(c[1]), int(c[2])
            index = (a << 5) | (b << 4) | (c_bit << 3) | (d_bit << 2) | (e_bit << 1) | f_bit
            score = scores[index]
            row.append(score)
            color_row.append(get_risk_color(score))
        table_data.append(row)
        cell_colors.append(color_row)

    table = ax.table(cellText=table_data, loc='center', cellLoc='center', cellColours=cell_colors)
    table.scale(1, 1.5)
    ax.set_title(f"Submap {idx+1} ({submap_labels[idx]})")

legend_elements = [
    Patch(facecolor='#a1d99b', edgecolor='black', label='Low Risk'),
    Patch(facecolor='#fec44f', edgecolor='black', label='Medium Risk'),
    Patch(facecolor='#de2d26', edgecolor='black', label='High Risk')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=12)

plt.show()

## How to Read the K-map

This K-map visualizes all possible combinations of the simplified **6-variable Morse Fall Scale**.

### Variables Encoding

| Variable | Bit                  | Meaning                                        |
| -------- | -------------------- | ---------------------------------------------- |
| A        | Most significant     | `falling_history` (1 if yes, 0 if no)          |
| B        | 2nd most significant | `secondary_diagnosis` (1 if yes, 0 if no)      |
| C        | 3rd                  | `high_ambulatory_aid` (1 if yes, 0 if no)      |
| D        | 4th                  | `iv_or_heparin_lock` (1 if yes, 0 if no)       |
| E        | 5th                  | `high_gait_or_transfering` (1 if yes, 0 if no) |
| F        | Least significant    | `does_forget_limitations` (1 if yes, 0 if no)  |

### Submaps

The map is split into 4 submaps based on the first two bits (A and B):

| Submap | A   | B   |
| ------ | --- | --- |
| 1      | 0   | 0   |
| 2      | 0   | 1   |
| 3      | 1   | 0   |
| 4      | 1   | 1   |

Each submap is an 8x8 grid representing the remaining four variables (C, D, E, F) using **Gray code ordering** for adjacency.

### Reading Cells

- Each cell shows the **Morse Fall Scale score** for that specific combination of variables.
- **Colors indicate risk level**:
  - **Green**: Low risk (score < 25)
  - **Yellow**: Medium risk (25 ≤ score ≤ 45)
  - **Red**: High risk (score > 45)

### Example

1. A cell in **Submap 2 (A=0, B=1)** with C=1, D=0, E=1, F=0:
    - Falling history: 0
    - Secondary diagnosis: 1
    - High ambulatory aid: 1
    - IV/Heparin lock: 0
    - High gait/transfering: 1
    - Forgets limitations: 0
2. The score is **75** (25 from B=1, 30 from C=1, 20 from E=1)
3. The cell is colored **red** indicating **High risk**

### Summary

- **Rows and columns follow Gray code**, so adjacent cells differ by only one variable.
- **Submap titles** indicate the fixed values of A and B.
- The **legend** shows the color coding for low, medium, and high risk.